In [1]:
import psycopg2
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')

# Connect to database
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD
)

# Helper function to run queries
def run_query(sql):
    return pd.read_sql(sql, conn)

print(f"Successfully connected to NGO database!")
print(f"Tables: assignments, donations, donars")

Successfully connected to NGO database!
Tables: assignments, donations, donars


![ngo_project_image](ngo_project_image.jpg)

GoodThought NGO has been a catalyst for positive change, focusing its efforts on education, healthcare, and sustainable development to make a significant difference in communities worldwide. With this mission, GoodThought has orchestrated an array of assignments aimed at uplifting underprivileged populations and fostering long-term growth.

This project offers a hands-on opportunity to explore how data-driven insights can direct and enhance these humanitarian efforts. In this project, you'll engage with the GoodThought PostgreSQL database, which encapsulates detailed records of assignments, funding, impacts, and donor activities from 2010 to 2023. This comprehensive dataset includes:

- **`Assignments`:** Details about each project, including its name, duration (start and end dates), budget, geographical region, and the impact score.
- **`Donations`:** Records of financial contributions, linked to specific donors and assignments, highlighting how financial support is allocated and utilized.
- **`Donors`:** Information on individuals and organizations that fund GoodThought’s projects, including donor types.

Refer to the below ERD diagram for a visual representation of the relationships between these data tables:
<img src="erd.png" alt="ERD" width="50%" height="50%">


You will execute SQL queries to answer two questions, as listed in the instructions. Good luck!


In [2]:
# Highest donation assignments by donor type
query = """
SELECT
    a.assignment_name,
    a.region,
    ROUND(SUM(d.amount)::numeric, 2) AS rounded_total_donation_amount,
    dn.donor_type
FROM
    donations d
JOIN assignments a ON d.assignment_id = a.assignment_id
JOIN donars dn ON d.donor_id = dn.donor_id
GROUP BY
    a.assignment_name, a.region, dn.donor_type
ORDER BY
    rounded_total_donation_amount DESC
LIMIT 5;
"""

df_highest = run_query(query)
print(f"Top 5 assignments by total donations (Total: {len(df_highest)} rows)")
display(df_highest)

Top 5 assignments by total donations (Total: 5 rows)


C:\Users\Asus\AppData\Local\Temp\ipykernel_8604\3715979485.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, conn)


,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


In [3]:
# Top regional impact assignments using window functions
query = """
WITH assignment_donations AS (
    SELECT
        a.assignment_id,
        a.assignment_name,
        a.region,
        a.impact_score,
        COUNT(d.donation_id) AS num_total_donations
    FROM
        assignments a
    JOIN donations d ON a.assignment_id = d.assignment_id
    GROUP BY
        a.assignment_id, a.assignment_name, a.region, a.impact_score
),
ranked_assignments AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY region ORDER BY impact_score DESC) AS rank
    FROM assignment_donations
)
SELECT
    assignment_name,
    region,
    impact_score,
    num_total_donations
FROM ranked_assignments
WHERE rank = 1
ORDER BY region ASC;
"""

df_top_regional = run_query(query)
print(f"Top impact assignment per region (Total: {len(df_top_regional)} regions)")
display(df_top_regional)

Top impact assignment per region (Total: 4 regions)


C:\Users\Asus\AppData\Local\Temp\ipykernel_8604\3715979485.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, conn)


,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,2
1,Assignment_2253,North,9.99,1
2,Assignment_3547,South,10.00,1
3,Assignment_3764,West,9.99,1
